In [1]:
#find somatic mutations with certain hugo ID
# UPDATE: THIS FILE NOW COMBINES ON COORDS, NOT HUGO
import csv
import os
import math
import statistics

with open("../../../coadreadDATLOWSPECIAL.csv","r") as f:
    it = csv.reader(f)
    listit = list(it)
with open("../../../pruned2DATSPECIAL.csv","r") as f:
    it = csv.reader(f)
    targetlistit = list(it)

print("feats in target: ",len(targetlistit[0])-2,"feats in listit: ",len(listit[0])-2)

samprecord = {}
seen = set()

#initialize samprecord. only new muts with these starting names can survive
for y in range(1,len(targetlistit[0])-1):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    CL = 0
    hiton = []
    for x in range(1, len(targetlistit)):
        if(float(targetlistit[x][y]) == 1.0):
            if (float(targetlistit[x][len(targetlistit[0])-1]) == -1.0):
                FP += 1
            elif (float(targetlistit[x][len(targetlistit[0])-1]) == 2.0):
                CL += 1
            else:
                TP += 1
                hiton.append(targetlistit[x][0])
        else:
            if (float(targetlistit[x][len(targetlistit[0])-1]) == -1.0):
                TN += 1
            else:
                FN += 1
        if(TP >= 3 or CL >= 2):
            samprecord[targetlistit[0][y]] = hiton

'''
#combine within pruned dataset first
for mut in targetlistit[0]:
    if mut == "" or mut == "class":
        continue
    pieces = mut.split("_")
    init = False
    for key in samprecord:
        pieces2 = key.split("_")
        if pieces[0] == pieces2[0]:
            if mut in seen:
                continue
            else:
                seen.add(mut)
            init = True
            #combine ontop of original
            #make new key
            newkey = key + "_" + "_".join(pieces[1:])
            samprecord[newkey] = samprecord.pop(key)
            #next append any new sample occurrences
            y = targetlistit[0].index(mut)
            for x in range(1,len(targetlistit)):
                if(float(targetlistit[x][y]) == 1.0):
                    if(targetlistit[x][0] not in samprecord[newkey]):
                        samprecord[newkey].append(targetlistit[x][0])
    if init == False:
        #make new key w sample occurrences
        samprecord[mut] = []
        y = targetlistit[0].index(mut)
        for x in range(1,len(targetlistit)):
            if(float(targetlistit[x][y]) == 1.0):
                samprecord[mut].append(targetlistit[x][0])
'''
            
samprecord

feats in target:  39 feats in listit:  88775


{'ACVR2A_GRCh37_2:148683686-148683693_Frame-Shift-Del-DEL-AAAAAAAA----': ['TCGA-AA-3518',
  'TCGA-AA-3672',
  'TCGA-AA-3710',
  'TCGA-AA-3715',
  'TCGA-AA-3821',
  'TCGA-AA-3833',
  'TCGA-AA-3845',
  'TCGA-AA-3877',
  'TCGA-AA-3947',
  'TCGA-AA-3949',
  'TCGA-AA-A01P',
  'TCGA-AA-A022'],
 'AIM2_GRCh37_1:159032487-159032496_Frame-Shift-Del-DEL-TTTTTTTTTT----': ['TCGA-AA-3715',
  'TCGA-AA-3821',
  'TCGA-AA-3947',
  'TCGA-AA-3949',
  'TCGA-AA-A00J',
  'TCGA-AA-A022'],
 'APC_GRCh37_5:112128143-112128143_Nonsense-Mutation-SNP-C-T-T': [],
 'APC_GRCh37_5:112173917-112173917_Nonsense-Mutation-SNP-C-C-T': [],
 'APC_GRCh37_5:112175423-112175423_Nonsense-Mutation-SNP-C-C-T': [],
 'APC_GRCh37_5:112175639-112175639_Nonsense-Mutation-SNP-C-C-T_Nonsense-Mutation-SNP-C-T-T': ['TCGA-AA-A00D',
  'TCGA-AA-A03F'],
 'ATR_GRCh37_3:142274740-142274749_Frame-Shift-Del-DEL-TTTTTTTTTT----': ['TCGA-AA-3672',
  'TCGA-AA-3715',
  'TCGA-AA-3821',
  'TCGA-AA-3949'],
 'BRAF_GRCh37_7:140453136-140453136_Missense-Mutat

In [2]:
#make our new dataset
combdatset = [[""]]
for key in samprecord:
    combdatset[0].append(key)
combdatset[0].append("class")

for x in range(1,len(targetlistit)):
    nextrow = [targetlistit[x][0]]
    for y in range(1,len(combdatset[0])-1):
        for key in samprecord:
            if key == combdatset[0][y]:
                if targetlistit[x][0] in samprecord[key]:
                    nextrow.append(1.0)
                else:
                    nextrow.append(0.0)
    nextrow.append(targetlistit[x][len(targetlistit[0])-1])
    combdatset.append(nextrow)
len(combdatset[0])-2

39

In [3]:
#combine onto the uncombined features from before pruning

#combine within pruned dataset first
for mut in listit[0]:
    if mut == "" or mut == "class":
        continue
    pieces = mut.split("_")
    init = False
    for key in samprecord:
        pieces2 = key.split("_")
        if pieces[0] + "_" + pieces[1] + "_" + pieces[2] == pieces2[0] + "_" + pieces2[1] + "_" + pieces2[2]:
            if mut in seen:
                continue
            else:
                seen.add(mut)
            init = True
            print("found one")
            #combine ontop of original
            #make new key
            newkey = key + "_" + "_".join(pieces[3:])
            samprecord[newkey] = samprecord.pop(key)
            #next append any new sample occurrences
            y = listit[0].index(mut)
            for x in range(1,len(listit)):
                if(float(listit[x][y]) == 1.0):
                    if(listit[x][0] not in samprecord[newkey]):
                        samprecord[newkey].append(listit[x][0])
    if init == False:
        continue

combdatset2 = [[""]]
for key in samprecord:
    combdatset2[0].append(key)
combdatset2[0].append("class")

for x in range(1,len(listit)):
    nextrow = [listit[x][0]]
    for y in range(1,len(combdatset2[0])-1):
        for key in samprecord:
            if key == combdatset2[0][y]:
                if listit[x][0] in samprecord[key]:
                    nextrow.append(1.0)
                else:
                    nextrow.append(0.0)
    nextrow.append(listit[x][len(listit[0])-1])
    combdatset2.append(nextrow)

len(samprecord)

found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one
found one


39

In [4]:
'''
with open("../../../pruned1DATSPECIALCOMBINED.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(combdatset)

with open("../../../pruned2DATSPECIALCOMBINED.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(combdatset2)
'''

'\nwith open("../../../pruned1DATSPECIALCOMBINED.csv","w") as fun:\n    writer = csv.writer(fun)\n    writer.writerows(combdatset)\n\nwith open("../../../pruned2DATSPECIALCOMBINED.csv","w") as fun:\n    writer = csv.writer(fun)\n    writer.writerows(combdatset2)\n'